# Genetic Algorithm Example Program

This is a simple illustration of a Genetic Algorithm finding the minimum value for the notorious all-ones problem, for the Natural Computing course at University of Edinburgh. This is by no means perfect code and should not be taken as such. The code was created by Billy Lyons, and takes some inspiration from https://github.com/ezstoltz/genetic-algorithm, with some changes for ease.

You can change the number of bits per string in the next cell. Other parameters enter the genetic_algorithm function directly, see last executable cell in the notebook.

In [1]:
NoB=50;

Importing necessary libraries

In [2]:
import numpy as np
import random
import operator
import matplotlib.pyplot as plt

Create initial population: In this formulation, the individual genes of our solutions consist of a string of 0s and 1s.

In [3]:
def initial_population(pop):
    population = []
    
    for i in range(pop):
        genome = []
        for i in range(NoB):
            genome.append(np.random.randint(0,2))
        population.append(genome)
    return population

# Fitness

In a genetic algorithm we need to know how good our individuals are for a number of reasons. If we are performing roulette wheel selection, we weight likelihood to breed and thus pass on genes by fitness, if we have some percentage of elitism, we may wish to keep the X most successful members of the population. As such we must define a fitness function for our routes.

In [4]:
def fitness(genome):
    
    fitness=sum(genome);
    
    return fitness

# Genetic Algorithm

We now have a population and a way to calculate fitness. We must use these to rank our population from best to worst. This is essential, as, you should weight the chance to reproduce by the fitness of each member, much in the same way that a better biological agent is more likely to pass on their genes with greater succes. Additionally, you may wish to maintain some elite members through to the next generation, so ordering is natural and makes life a lot easier.

In [5]:
def ranking_pop(population):
    fitnesses = {}
    sum_fit = 0.0
    for i in range(len(population)):
        fitnesses[i] = fitness(population[i])
        sum_fit += fitnesses[i]
    return sorted(fitnesses.items(), key=operator.itemgetter(1),reverse=True), sum_fit

Now that we have our sorted list, we must perform selection for the next parents of the next generation. Here we are going to replicate into the next generation a certain number of elite agents, and we take the population size minus this elitism, and draw that many members from the generation by a probability distribution which is determined by their individual fitness.

1) Why might we want to have some level of elitism?

2) Why do we use a weighted selection?

In [6]:
def selection(ranked, tot, elitism):
    probabilities = np.zeros(len(ranked))
    members = np.arange(len(ranked))
    size = len(ranked)
    elite_members = []
    for i in range(len(ranked)):
        x,y = ranked[i][0], ranked[i][1]
        if i < elitism:
            elite_members.append(x)
        probabilities[x] = y
    probabilities=probabilities/np.sum(probabilities)
    selected_members = np.random.choice(members, size-elitism, p=probabilities)
    return selected_members, elite_members

In [7]:
def mating_pool(population, selected_members, elite_members):
    mating_pool = []
    elite = []
    for i in range(len(selected_members)):
        index = selected_members[i]
        mating_pool.append(population[index])
                
    for i in range(len(elite_members)):
        index = elite_members[i]
        elite.append(population[index])        
    return mating_pool, elite

Now that we have our pool of viable mates, we randomly draw some beginning and end point for the crossover between the two chromosomes. Maintaining order.

1) Look at the below, what is happening? How is this different from point crossover in the all ones problem and the class notes?

2) Why is it important to maintaining ordering?

3) Crossover here occurs every time. How might you change this to a probabilistic method? Why might that be better/worse?


In [8]:
def breed_from_parents(parent1, parent2):
    geneA = int(np.random.rand()*len(parent1))
    geneB = int(np.random.rand()*len(parent1))

    while geneB == geneA:
        geneB = int(np.random.rand()*len(parent1))
    
    start = min(geneA, geneB)
    end = max(geneA, geneB)

    child = []
    for i in range(len(parent1)):
        if i>=start and i<end:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child

In [9]:
def new_population(mating_pool, elite_pool):
    children = []
    pool = random.sample(mating_pool, len(mating_pool))
        
    for i in range(len(elite_pool)):
        children.append(elite_pool[i])
    for i in range(len(mating_pool)):
        child = breed_from_parents(pool[i], pool[len(mating_pool)-i-1])
        children.append(child)
    return children

Now that our parent population has breeded and we have generated our new population from the children and the elites who have lived into the next generation, we must go through each solution and check to see if any must be mutated. We are doing this by running through each candidate in the new population, and at each point in the chromosome we see if it will mutate here and then perform a swap.

1) Why is mutation an important part of a GA (hint: Think about the search space)?

2) If you had a chromosome reprsenting some real number e.g. 3.14159 -> chromosome 314159, how might you adapt the mutation rate?


In [10]:
def mutate(chromosome, prob_mut):
    for mutable in range(len(chromosome)):
        if (np.random.rand()<prob_mut):
            chromosome[mutable]=1-chromosome[mutable]
    return chromosome

In [11]:
def mutation_over_pop(population, prob_mut):
    mutated_pop = []
    for i in range(len(population)):
        mutated_chromo = mutate(population[i], prob_mut)
        mutated_pop.append(mutated_chromo)
    return mutated_pop

In [12]:
def new_generation(current_gen, elitism, prob_mut):
    rank, tot = ranking_pop(current_gen)
    selected_members, elite_members = selection(rank, tot, elitism)
    mates, elites = mating_pool(current_gen, selected_members, elite_members)
    children = new_population(mates, elites)
    next_gen = mutation_over_pop(children, prob_mut)
    return next_gen    

Now we put everything together and run it for the generations!

In [13]:
def genetic_algorithm(population, elitism, prob_mut, generations):
    pop = initial_population(population)
    ranked, tot = ranking_pop(pop)
    #print(ranked)
    print("Initial best fitness: {}".format(ranked[0][1]))
    
    for i in range(generations):
        pop = new_generation(pop, elitism, prob_mut)
        ranked, tot = ranking_pop(pop)
        if ranked[0][1]==NoB:
            break;
    
    ranked, tot = ranking_pop(pop)
    print("Final best fitness: {}".format(ranked[0][1]))
    
    return i

In [14]:
zz=genetic_algorithm(50, 1, 0.01, 1000)
print("run time: ",zz)

Initial best fitness: 33
Final best fitness: 50
run time:  986


# Next

Many things could be improved here, but we should move on.

Have a look at the TSP GA, next.